## Classifying digits with convolutional neural networks

This notebook contains the solution to the MNIST activity. 

#### Load the data

Both Keras and TF-Learn contain the MNIST dataset that can be quickly loaded with some helper functions. This solution will use TF-Learn but the Keras solution will be commented out. The two libraries are very similar. 

In [1]:
import tflearn
import tflearn.datasets.mnist as mnist

# Load data from TF-Learn 
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])


Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz


/Users/florianmuellerklein/anaconda/envs/tensorflow/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
/Users/florianmuellerklein/anaconda/envs/tensorflow/lib/python2.7/site-packages/tflearn/datasets/mnist.py:52: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz


#### Build the ConvNet 

Create a small convolutional that will run on a CPU, so only use about 6 and 8 kernels in each convolutional layer. For the fully connected layer, just use a 32 to 64 units as well. We won't get state of the art performance but we don't want to wait all day for it to run. 

In [2]:
# -------------------------------------------------------------------------------------------
# TF-Learn
# build the network
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

# specify the size of the input data
net = input_data(shape=[None, 28, 28, 1], name='input')
# for each layer, we pass (name_input_data, number of kernels, kernel size, activation function)
net = conv_2d(net, 6, 3, activation='relu')
net = max_pool_2d(net, 2)

net = conv_2d(net, 8, 3, activation='relu')
net = max_pool_2d(net, 2)

net = fully_connected(net, 32, activation='relu')
net = dropout(net, 0.5)

net = fully_connected(net, 10, activation='softmax')
net = regression(net, optimizer='adam', learning_rate=0.001, 
                 loss='categorical_crossentropy', name='target')

# -------------------------------------------------------------------------------------------
# Keras
#import keras
#from keras.models import Model
#from keras.optimizers import Adam
#from keras.layers import Input, Dense, Activation, Dropout, Flatten, merge
#from keras.layers.convolutional import Convolution2D, MaxPooling2D

#keras.callbacks.TensorBoard(log_dir='/tmp/tflearn_logs', histogram_freq=0, write_graph=True, write_images=False)


#cnn_input = Input(shape=(28, 28, 1), name='Input')

#net = Convolution2D(6, 3,3, activation='relu')(cnn_input)
#net = MaxPooling2D(pool_size=(2,2))(net)

#net = Convolution2D(8, 3,3, activation='relu')(net)
#net = MaxPooling2D(pool_size=(2,2))(net)

# keras uses a flatten layer when going from convolutional layers to normal
#net = Flatten()(net)

#net = Dense(32, activation='relu')(net)
#net = Dropout(p=0.5)(net)

#out = Dense(10, activation='softmax')(net)
#model = Model(input=cnn_input, output=out)
#adam = Adam(lr=0.001)
#model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

#### Train the network

Train the network and use the test data as the validation set. 

In [3]:
# Tf-Learn train
model = tflearn.DNN(net, tensorboard_verbose=1)
model.fit({'input': X}, {'target': Y}, n_epoch=20,
           validation_set=({'input': testX}, {'target': testY}),
           snapshot_step=100, show_metric=True, run_id='convnet_mnist')

# when training the tf-learn model go to http://192.168.1.27:6006

# Keras Train
#model.fit(X, Y)

Training Step: 17200  | total loss: 0.55870
| Adam | epoch: 020 | loss: 0.55870 - acc: 0.9208 | val_loss: 0.05927 - val_acc: 0.9810 -- iter: 55000/55000
Training Step: 17200  | total loss: 0.55870
| Adam | epoch: 020 | loss: 0.55870 - acc: 0.9208 | val_loss: 0.05927 - val_acc: 0.9810 -- iter: 55000/55000
--
